In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

/Users/adishsundar/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
data = pd.read_stata(r"/Users/adishsundar/Desktop/Salem Center Research/113482-V1/Davila_Mora_data.dta")
df = data
df.head(3)

,year,statefip,puma,perwt,age,hispan,bpl,language,speakeng,schl,labforce,occ,hisp,mexam,prican,cuban,othhisp,prborn,imm,female,lep,inpov,educ,exper,exp2,coh90_2534,coh00_2534,engindex,cogdisable,notcit,veteran,mexborn,cubborn,nextdecade,educ10,exper10,fem10,i1,i3,i310,year00,year10,exp210,neng,midatl,soatl,ncent,scent,mountain,pacific,outmsa,exec,healthtech,offadmin,service,sales,farmag,milit,bluecollar,salesserv,occcat,noocc,nextdecbothcoh,neng10,midatl10,soatl10,ncent10,scent10,mountain10,pacific10,exec10,healthtech10,offadm10,salesserv10,farm10,blue10,milit10,noocc10,prican10,cuban10,pricanlep,cubanlep,spanishshare,spanishshare10,mexam10,mexlep,notlep
0,2013,Alabama,100,589.0,33,Mexican,Mexico,Spanish,"Yes, but not well",Regular high school diploma,"Yes, in the labor force",6260,1,1,0,0,0,0,1,0,1,0.0,12.0,16.0,256.0,0,0,0.33,0,1,0,1,0,0,0.0,0.0,0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,0
1,2000,Alabama,100,31.0,42,Mexican,Mexico,Spanish,"Yes, speaks very well","1 or more years of college credit, no degree","Yes, in the labor force",31,1,1,0,0,0,0,1,0,0,0.0,13.0,24.0,576.0,1,0,1.00,0,0,0,1,0,1,13.0,24.0,0,0,0,0,1,0,576.0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.435148,1.435148,1,0,1
2,2013,Alabama,100,478.0,39,Mexican,Mexico,Spanish,Does not speak English,Grade 6,"Yes, in the labor force",8965,1,1,0,0,0,0,1,1,1,0.0,6.0,28.0,784.0,0,0,0.00,0,1,0,1,0,0,0.0,0.0,0,0,0,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,0


## REPLICATION

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Load your data
# Assuming the DataFrame is named df
# df = pd.read_csv('path_to_your_data.csv')

# Filter data for the first model (probit)
df_probit = df[(df['year'] == 2013) & (df['age'] >= 25) & (df['age'] <= 64) & (df['milit'] == 0) & 
              ((df['cuban'] == 1) & (df['imm'] == 1) | 
               (df['prican'] == 1) & (df['prborn'] == 1) | 
               (df['mexam'] == 1) & (df['imm'] == 1))]

# Define the probit model
probit_formula = 'cogdis ~ mexam + cuban + mexlep + cubanlep + lep + educ + exper + exp2 + female + i1 + neng + midatl + soatl + ncent + scent + mountain + healthtech + offadmin + farmag + exec + salesserv + noocc + veteran + notcit'
probit_model = smf.probit(probit_formula, data=df_probit, weights=df_probit['perwt'])
probit_results = probit_model.fit()

# Print results
print(probit_results.summary())

TypeError: Invalid comparison between dtype=category and int

In [6]:
# Filter data for the ordered probit model
df_oprobit = df[((df['mexam'] == 1) & (df['imm'] == 1) | 
                 (df['cuban'] == 1) & (df['imm'] == 1) | 
                 (df['prican'] == 1) & (df['prborn'] == 1)) & 
                ((df['year'] == 1990) | (df['year'] == 2000)) & 
                (df['coh90_2534'] == 1) & (df['milit'] == 0)]

# Define the ordered probit model
oprobit_formula = 'engindex ~ nextdecade + mexam + cuban + mexam10 + cuban10 + educ + educ10 + spanishshare + spanishshare10 + exper + exper10 + exp2 + exp210 + female + fem10 + i1 + i310 + healthtech + offadmin + salesserv + farmag + noocc + exec + healthtech10 + offadm10 + salesserv10 + farm10 + exec10 + noocc10'
oprobit_model = smf.ordered_probit(oprobit_formula, data=df_oprobit, weights=df_oprobit['perwt'])
oprobit_results = oprobit_model.fit()

# Print results
print(oprobit_results.summary())

AttributeError: module 'statsmodels.formula.api' has no attribute 'ordered_probit'

In [10]:
# Convert necessary categorical columns to integers
df['year'] = df['year'].astype(int)
df['age'] = df['age'].astype(int)
df['milit'] = df['milit'].astype(int)
df['cuban'] = df['cuban'].astype(int)
df['imm'] = df['imm'].astype(int)
df['prican'] = df['prican'].astype(int)
df['prborn'] = df['prborn'].astype(int)
df['mexam'] = df['mexam'].astype(int)

# Continue with the filtering as before
df_probit = df[
    (df['year'] == 2013) & (df['age'] >= 25) & (df['age'] <= 64) & (df['milit'] == 0) &
    ((df['cuban'] == 1) & (df['imm'] == 1) |
     (df['prican'] == 1) & (df['prborn'] == 1) |
     (df['mexam'] == 1) & (df['imm'] == 1))
]

# Check for any other categorical variables that may need conversion
# And similarly prepare df_oprobit
df_oprobit = df[
    ((df['mexam'] == 1) & (df['imm'] == 1) |
     (df['cuban'] == 1) & (df['imm'] == 1) |
     (df['prican'] == 1) & (df['prborn'] == 1)) &
    ((df['year'] == 1990) | (df['year'] == 2000)) &
    (df['coh90_2534'] == 1) & (df['milit'] == 0)
]

# Define and fit your models as planned
probit_formula = 'cogdis ~ mexam + cuban + mexlep + cubanlep + lep + educ + exper + exp2 + female + i1 + neng + midatl + soatl + ncent + scent + mountain + healthtech + offadmin + farmag + exec + salesserv + noocc + veteran + notcit'
probit_model = smf.probit(probit_formula, data=df_probit, weights=df_probit['perwt'])
probit_results = probit_model.fit()
print(probit_results.summary())

# For the ordered probit, define the model and fit it
oprobit_formula = 'engindex ~ nextdecade + mexam + cuban + mexam10 + cuban10 + educ + educ10 + spanishshare + spanishshare10 + exper + exper10 + exp2 + exp210 + female + fem10 + i1 + i310 + healthtech + offadmin + salesserv + farmag + noocc + exec + healthtech10 + offadm10 + salesserv10 + farm10 + exec10 + noocc10'
oprobit_model = smf.ordered_probit(oprobit_formula, data=df_oprobit, weights=df_oprobit['perwt'])
oprobit_results = oprobit_model.fit()
print(oprobit_results.summary())

PatsyError: Error evaluating factor: NameError: name 'cogdis' is not defined
    cogdis ~ mexam + cuban + mexlep + cubanlep + lep + educ + exper + exp2 + female + i1 + neng + midatl + soatl + ncent + scent + mountain + healthtech + offadmin + farmag + exec + salesserv + noocc + veteran + notcit
    ^^^^^^

In [11]:
# Correct the formula to use 'cogdisable' instead of 'cogdis'
probit_formula = 'cogdisable ~ mexam + cuban + mexlep + cubanlep + lep + educ + exper + exp2 + female + i1 + neng + midatl + soatl + ncent + scent + mountain + healthtech + offadmin + farmag + exec + salesserv + noocc + veteran + notcit'

# Define and fit the probit model
probit_model = smf.probit(probit_formula, data=df, weights=df['perwt'])
probit_results = probit_model.fit()
print(probit_results.summary())

/Users/adishsundar/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:130: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.345569
         Iterations 6


/Users/adishsundar/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:130: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


                          Probit Regression Results                           
Dep. Variable:             cogdisable   No. Observations:               559985
Model:                         Probit   Df Residuals:                   559960
Method:                           MLE   Df Model:                           24
Date:                Sat, 04 May 2024   Pseudo R-squ.:                 0.03720
Time:                        19:18:25   Log-Likelihood:            -1.9351e+05
converged:                       True   LL-Null:                   -2.0099e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8081      0.019    -42.037      0.000      -0.846      -0.770
mexam         -0.1944      0.011    -17.601      0.000      -0.216      -0.173
cuban         -0.0928      0.014     -6.843      0.0

In [12]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Example data loading step (ensure you've loaded your data correctly)
# df = pd.read_csv('your_data_file.csv')

# Convert necessary categorical columns to integers
df['year'] = df['year'].astype(int)
df['age'] = df['age'].astype(int)
df['milit'] = df['milit'].astype(int)
df['cuban'] = df['cuban'].astype(int)
df['imm'] = df['imm'].astype(int)
df['prican'] = df['prican'].astype(int)
df['prborn'] = df['prborn'].astype(int)
df['mexam'] = df['mexam'].astype(int)

# Filtering for the probit model
df_probit = df[
    (df['year'] == 2013) & (df['age'] >= 25) & (df['age'] <= 64) & (df['milit'] == 0) &
    ((df['cuban'] == 1) & (df['imm'] == 1) |
     (df['prican'] == 1) & (df['prborn'] == 1) |
     (df['mexam'] == 1) & (df['imm'] == 1))
]

# Define the probit model with the corrected variable name
probit_formula = 'cogdisable ~ mexam + cuban + mexlep + cubanlep + lep + educ + exper + exp2 + female + i1 + neng + midatl + soatl + ncent + scent + mountain + healthtech + offadmin + farmag + exec + salesserv + noocc + veteran + notcit'
probit_model = smf.probit(probit_formula, data=df_probit, weights=df_probit['perwt'])
probit_results = probit_model.fit()

# Filtering for the ordered probit model
df_oprobit = df[
    ((df['mexam'] == 1) & (df['imm'] == 1) |
     (df['cuban'] == 1) & (df['imm'] == 1) |
     (df['prican'] == 1) & (df['prborn'] == 1)) &
    ((df['year'] == 1990) | (df['year'] == 2000)) &
    (df['coh90_2534'] == 1) & (df['milit'] == 0)
]

# Define the ordered probit model
oprobit_formula = 'engindex ~ nextdecade + mexam + cuban + mexam10 + cuban10 + educ + educ10 + spanishshare + spanishshare10 + exper + exper10 + exp2 + exp210 + female + fem10 + i1 + i310 + healthtech + offadmin + salesserv + farmag + noocc + exec + healthtech10 + offadm10 + salesserv10 + farm10 + exec10 + noocc10'
oprobit_model = smf.ordered_probit(oprobit_formula, data=df_oprobit, weights=df_oprobit['perwt'])
oprobit_results = oprobit_model.fit()

# Print results
print("Probit Model Summary:")
print(probit_results.summary())
print("\nOrdered Probit Model Summary:")
print(oprobit_results.summary())

/Users/adishsundar/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:130: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.149855
         Iterations 7


AttributeError: module 'statsmodels.formula.api' has no attribute 'ordered_probit'

# RESULTS

### The results below can be compared to Table 3 in the left column for the Ordered probit Model

In [15]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

# Define the independent variables and the dependent variable for the Ordered Probit model
independent_vars = df_oprobit[['nextdecade', 'mexam', 'cuban', 'mexam10', 'cuban10', 'educ', 'educ10', 'spanishshare', 'spanishshare10', 'exper', 'exper10', 'exp2', 'exp210', 'female', 'fem10', 'i1', 'i310', 'healthtech', 'offadmin', 'salesserv', 'farmag', 'noocc', 'exec', 'healthtech10', 'offadm10', 'salesserv10', 'farm10', 'exec10', 'noocc10']]
dependent_var = df_oprobit['engindex']

# Fit the Ordered Probit model
oprobit_model = OrderedModel(dependent_var, independent_vars, distr='probit')
oprobit_results = oprobit_model.fit(method='bfgs', disp=True)  # Use BFGS method, typically good for this kind of model

print("\nOrdered Probit Model Summary:")
print(oprobit_results.summary())

Optimization terminated successfully.
         Current function value: 1.174055
         Iterations: 144
         Function evaluations: 149
         Gradient evaluations: 149

Ordered Probit Model Summary:
                             OrderedModel Results                             
Dep. Variable:               engindex   Log-Likelihood:            -1.1769e+05
Model:                   OrderedModel   AIC:                         2.354e+05
Method:            Maximum Likelihood   BIC:                         2.357e+05
Date:                Sat, 04 May 2024                                         
Time:                        19:37:07                                         
No. Observations:              100239                                         
Df Residuals:                  100207                                         
Df Model:                          29                                         
                     coef    std err          z      P>|z|      [0.025      0.975]


### The results below can be compared to results reported in the text in Section I

In [16]:
# Fit the Probit model (make sure df_probit is correctly filtered and prepared)
probit_model = smf.probit('cogdisable ~ mexam + cuban + mexlep + cubanlep + lep + educ + exper + exp2 + female + i1 + neng + midatl + soatl + ncent + scent + mountain + healthtech + offadmin + farmag + exec + salesserv + noocc + veteran + notcit', data=df_probit, weights=df_probit['perwt'])
probit_results = probit_model.fit()
print("Probit Model Summary:")
print(probit_results.summary())

/Users/adishsundar/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:130: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.149855
         Iterations 7
Probit Model Summary:


/Users/adishsundar/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:130: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


                          Probit Regression Results                           
Dep. Variable:             cogdisable   No. Observations:                83462
Model:                         Probit   Df Residuals:                    83437
Method:                           MLE   Df Model:                           24
Date:                Sat, 04 May 2024   Pseudo R-squ.:                  0.1572
Time:                        19:37:09   Log-Likelihood:                -12507.
converged:                       True   LL-Null:                       -14841.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8687      0.075    -24.870      0.000      -2.016      -1.721
mexam         -0.4716      0.038    -12.399      0.000      -0.546      -0.397
cuban         -0.1864      0.046     -4.026      0.0